In [1]:
import pandas as pd
import csv
import os
import numpy as np
import cv2
pd.options.mode.chained_assignment = None

In [2]:
def read_data_with_subdirectorys(data_path, extension):
    videos_path_list = []
    print("List of all directories in '% s':" % data_path)

    for path, subdirs, files in os.walk(data_path):
        for name in files:
            if(extension is not None):
                if name.endswith(extension):
                    videos_path_list.append(os.path.join(path, name))
            else:
                videos_path_list.append(os.path.join(path, name))

    return videos_path_list

def is_au(string):
    txt = string
    #print("@@@@@@@@@@@@@@@@@@@@")
    #print(txt)
    if(txt):
        x = txt.replace("+", "")
        return x.isdigit()
    else:
        #print("@@@@@@@@@@@@@@@@@@@@")
        print(txt)
        return False


def read_concat_xml(path):
    
    df_results = pd.DataFrame(columns=['frame', 'start', 'end', 'video_name'])
    
    lista_xmls = read_data_with_subdirectorys(path, '.xml')
    file = None
    for file in lista_xmls:
        print(file)
        df = pd.read_xml(file)
        df = df.reset_index() #para pegar o frame
        df['index'] = df['index'] +1
        df = df.rename(columns={'index':'frame', 'v':'aus'})
        df['video_name'] = file.split('/')[-1].split('.')[0]
        print(df.shape)
        #print(df_results.shape)
        
        df_results = pd.concat([df_results, df], ignore_index=True)

    return df_results

In [3]:
path_annotations = "../data/raw/HM-Libras/Annotations/"
path_videos = '../data/raw/HM-Libras/Videos/'
df = read_concat_xml(path_annotations)
df = df.sort_values(by=['video_name', 'start'], ascending=(False, True))
df

List of all directories in '../data/raw/HM-Libras/Annotations/':
../data/raw/HM-Libras/Annotations/S06TF04V02.xml
(91, 5)
../data/raw/HM-Libras/Annotations/S08TF02V01.xml
(81, 5)
../data/raw/HM-Libras/Annotations/S04TF01V01.xml
(37, 5)
../data/raw/HM-Libras/Annotations/S09TF02V01.xml
(31, 5)
../data/raw/HM-Libras/Annotations/S07TF01V01.xml
(84, 5)
../data/raw/HM-Libras/Annotations/S08TF03V02.xml
(54, 5)
../data/raw/HM-Libras/Annotations/S02TF02V01.xml
(73, 5)
../data/raw/HM-Libras/Annotations/S07TF02V01.xml
(67, 5)
../data/raw/HM-Libras/Annotations/S09TF03V01.xml
(16, 5)
../data/raw/HM-Libras/Annotations/S05TF03V01.xml
(39, 5)
../data/raw/HM-Libras/Annotations/S06TF02V02.xml
(28, 5)
../data/raw/HM-Libras/Annotations/S04TF04V02.xml
(38, 5)
../data/raw/HM-Libras/Annotations/S06TF01V02.xml
(43, 5)
../data/raw/HM-Libras/Annotations/S05TF04V02.xml
(24, 5)
../data/raw/HM-Libras/Annotations/S02TF03V02.xml
(37, 5)
../data/raw/HM-Libras/Annotations/S07TF03V01.xml
(58, 5)
../data/raw/HM-Libras/A

,frame,start,end,video_name,aus
900,1,0.066,0.300,S10TF01V01,12+25+26
901,2,0.066,6.794,S10TF01V01,Claro! É o meu sonho!
902,3,0.300,0.333,S10TF01V01,12+25+26+42+51
903,4,0.333,0.533,S10TF01V01,12+25+26+43+51
904,5,0.533,0.567,S10TF01V01,15+25+26+42+51
...,...,...,...,...,...
2657,30,5.293,5.393,S01TF01V01,12+25+43
2658,31,5.393,5.493,S01TF01V01,12+25
2659,32,5.493,5.727,S01TF01V01,25+26
2660,33,5.727,5.760,S01TF01V01,25+26+62


In [4]:
def define_frame_init_final(df3, fps_video):

    df3['frame_inicial'] = np.ceil((df3['start'])*fps_video).astype(int)
    #df3['frame_final'] = df3['frame_inicial'].shift(-1, fill_value=0)
    df3['frame_final'] = np.ceil((df3['end'])*fps_video).astype(int)
    df3['frame_final'] = df3['frame_final'].apply(lambda x: x-1 if x>0 else x) #diminundo 1 frame para evitar sobreposição

    #Eliminando a ocorrência com frase
    df3["au_valid"] = df3.apply(lambda row: is_au(row['aus']), axis=1) 
    df3 = df3.loc[df3.au_valid == True][['frame', 'start', 'end', 'video_name', 'aus', 'frame_inicial', 'frame_final']]
    
    return df3

def aus_distribution(df3):
    df_true = pd.DataFrame(columns = ['video_name', 'frame', 'aus'])
    frame_max = df3.frame_final.max()
    frame_min = df3.frame_final.min()
    #print(frame_max, str(frame_max))
    #print(frame_min, str(frame_min))

    for i in range(0, frame_max):
        row = df3.loc[(df3.frame_inicial <= i+1) & (df3.frame_final >= i+1)]
        
        #display(row)
        
        if(len(row) ==1):
            #print("iguaaaaaaaaaaaaaaaaaaaaaaal a 1 row: ", " /video_name "  , row.video_name.unique().item(), " /frame ", i+1, " /aus ", row.aus.item())

            new_row = {
                "video_name": row.video_name.item(),
                "frame": i+1,
                "aus": row.aus.item(),
            }

            df_true = df_true.append(new_row, ignore_index=True)
            
        # No HM-LIBRAS existem casos em que existem duas rows com Unidades de ação para um mesmo intervalo de tempo...
        # Portanto, iremos unificá-las
        # Caso de teste: df.loc[df.video_name == 'S06TF01V01']
        
        elif(len(row) > 1):
            
            aus_list = row.aus.values.flatten().tolist()
            aus_list = list(set(aus_list))
            my_lst_str = '+'.join(map(str, aus_list))
            
            #print("maiorrrrrrrrrrrrrrrrrrrr que 1 row: ", " /video_name ", row.video_name.unique().item()," /frame ", i+1, " /aus ",my_lst_str)
            new_row = {
                "video_name": row.video_name.unique().item(),
                "frame": i+1,
                "aus": my_lst_str,
            }

            df_true = df_true.append(new_row, ignore_index=True)
        else:
            #print("zerooooooooooooo")
            pass
    return df_true
    


In [5]:
t = [1, 2, 3, 1, 2, 5, 6, 7, 8, 8]

list(set(t))

[1, 2, 3, 5, 6, 7, 8]

In [6]:
my_lst_str = '+'.join(map(str, t))
my_lst_str

'1+2+3+1+2+5+6+7+8+8'

In [7]:
df

,frame,start,end,video_name,aus
900,1,0.066,0.300,S10TF01V01,12+25+26
901,2,0.066,6.794,S10TF01V01,Claro! É o meu sonho!
902,3,0.300,0.333,S10TF01V01,12+25+26+42+51
903,4,0.333,0.533,S10TF01V01,12+25+26+43+51
904,5,0.533,0.567,S10TF01V01,15+25+26+42+51
...,...,...,...,...,...
2657,30,5.293,5.393,S01TF01V01,12+25+43
2658,31,5.393,5.493,S01TF01V01,12+25
2659,32,5.493,5.727,S01TF01V01,25+26
2660,33,5.727,5.760,S01TF01V01,25+26+62


In [8]:
df.dtypes

frame          object
start         float64
end           float64
video_name     object
aus            object
dtype: object

In [9]:
df['frame'] = df['frame'].astype(int)
df.dtypes

frame           int64
start         float64
end           float64
video_name     object
aus            object
dtype: object

In [10]:
df.loc[df.video_name == 'S06TF01V01']

,frame,start,end,video_name,aus
1284,1,0.08,0.160,S06TF01V01,43
1285,2,0.16,0.280,S06TF01V01,53
1286,3,0.16,0.400,S06TF01V01,10
1287,4,0.28,0.480,S06TF01V01,12+25
1288,5,0.48,0.800,S06TF01V01,1+2+25+26
1289,6,0.56,5.579,S06TF01V01,Filme de ação
1290,7,0.80,1.040,S06TF01V01,1+2
1291,8,1.04,1.120,S06TF01V01,1+2+25+26
1292,9,1.12,1.200,S06TF01V01,1+2+23+25
1293,10,1.20,1.280,S06TF01V01,1+2+23


In [24]:
df_results = pd.DataFrame(columns = ['video_name', 'frame', 'aus'])

for i in df.video_name.unique(): #Cse .loc[df.video_name == 'S06TF02V02']
        extensao = '.mp4'
        print(path_videos+i+extensao)
        v_entry = cv2.VideoCapture(path_videos+i+extensao, 0)
        #frames = int(v_entry.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = v_entry.get(cv2.CAP_PROP_FPS)
        df_frames = define_frame_init_final(df.loc[df.video_name == i], fps)
        #display(df_frames)
        #print("@@@@@@@@@@@")
        df_aus = aus_distribution(df_frames)
        #print("uuuuuuuuuuuuuuuuu")
        #display(df_aus)
        
        df_results = pd.concat([df_aus, df_results], ignore_index=True)

display(df_results)        

../data/raw/HM-Libras/Videos/S10TF01V01.mp4
../data/raw/HM-Libras/Videos/S09TF04V02.mp4
../data/raw/HM-Libras/Videos/S09TF04V01.mp4
../data/raw/HM-Libras/Videos/S09TF03V02.mp4
../data/raw/HM-Libras/Videos/S09TF03V01.mp4
../data/raw/HM-Libras/Videos/S09TF02V01.mp4
../data/raw/HM-Libras/Videos/S09TF01V02.mp4
../data/raw/HM-Libras/Videos/S09TF01V01.mp4
../data/raw/HM-Libras/Videos/S08TF04V02.mp4
../data/raw/HM-Libras/Videos/S08TF04V01.mp4
../data/raw/HM-Libras/Videos/S08TF03V02.mp4
../data/raw/HM-Libras/Videos/S08TF03V01.mp4
../data/raw/HM-Libras/Videos/S08TF02V02.mp4
../data/raw/HM-Libras/Videos/S08TF02V01.mp4
None
None
None
../data/raw/HM-Libras/Videos/S08TF01V02.mp4
../data/raw/HM-Libras/Videos/S08TF01V01.mp4
../data/raw/HM-Libras/Videos/S07TF04V02.mp4
None
../data/raw/HM-Libras/Videos/S07TF04V01.mp4
../data/raw/HM-Libras/Videos/S07TF03V02.mp4
../data/raw/HM-Libras/Videos/S07TF03V01.mp4
../data/raw/HM-Libras/Videos/S07TF02V02.mp4
../data/raw/HM-Libras/Videos/S07TF02V01.mp4
../data/raw/

,video_name,frame,aus
0,S01TF01V01,1,4
1,S01TF01V01,2,4
2,S01TF01V01,3,25+26
3,S01TF01V01,4,25+26
4,S01TF01V01,5,25+26
...,...,...,...
13112,S10TF01V01,204,12+25+26
13113,S10TF01V01,205,12+25+26
13114,S10TF01V01,206,12+25+26
13115,S10TF01V01,207,12+25+26


In [25]:
len(df_results.video_name.unique())

72

In [26]:
df_results.video_name.value_counts()

S06TF04V02    501
S06TF02V01    479
S02TF02V01    442
S05TF01V01    388
S02TF02V02    358
             ... 
S09TF04V01     69
S05TF03V02     65
S06TF04V01     61
S08TF04V02     60
S03TF03V01     52
Name: video_name, Length: 72, dtype: int64

In [27]:
df_results.loc[df_results.video_name == 'S06TF01V01'].head(30)

,video_name,frame,aus
6812,S06TF01V01,2,43
6813,S06TF01V01,3,43
6814,S06TF01V01,4,10+53
6815,S06TF01V01,5,10+53
6816,S06TF01V01,6,10+53
6817,S06TF01V01,7,10+53
6818,S06TF01V01,8,12+25+10
6819,S06TF01V01,9,12+25+10
6820,S06TF01V01,10,12+25
6821,S06TF01V01,11,12+25


In [21]:
df_results.loc[df_results.video_name == 'S05TF03V02'].head(30)

,video_name,frame,aus
6527,S05TF03V02,2,23+52
6528,S05TF03V02,3,23+52
6529,S05TF03V02,4,23+52
6530,S05TF03V02,5,23+52
6531,S05TF03V02,6,23+52
6532,S05TF03V02,7,23+52
6533,S05TF03V02,8,23+52
6534,S05TF03V02,9,23+52
6535,S05TF03V02,10,23+43+52
6536,S05TF03V02,11,23+43+52


In [22]:
df_results = df_results.sort_values(by=['video_name', 'frame'], ascending=(False, True))
df_results

,video_name,frame,aus
12910,S10TF01V01,2,12+25+26
12911,S10TF01V01,3,12+25+26
12912,S10TF01V01,4,12+25+26
12913,S10TF01V01,5,12+25+26
12914,S10TF01V01,6,12+25+26
...,...,...,...
158,S01TF01V01,171,25+26
159,S01TF01V01,172,25+26+62
160,S01TF01V01,173,4+25+26+62
161,S01TF01V01,174,4+25+26+62


In [23]:
df_results = df_results.rename(columns={"aus": "ground truth"})
df_results.to_csv("../data/outputs/HM-Libras/predicts_squeezenet/HM-Libras-ground-truth.csv", sep=';')